In [1]:
import json
import requests as r
import sqlalchemy as sqla
import pandas as pd
import csv

PROXIES = {
 "http": "http://07343957:X@nd@0l@bs0734@proxy-1dn.mb:6060",
 "https": "http://07343957:X@nd@0l@bs0734@proxy-1dn.mb:6060",
}

url_base = 'http://compras.dados.gov.br'

def get_pregoes(uasg: str):
    data = {'co_uasg': str(uasg)}
    href_pregoes = '/pregoes/v1/pregoes.json'
    url = url_base + href_pregoes
    response = r.request('GET', url=url, params=data, proxies=PROXIES)
    return response.json()


In [2]:
df = pd.DataFrame.from_dict(get_pregoes('762200')['_embedded']['pregoes'])

In [3]:
df.iloc[0]['_links']

{'self': {'href': '/pregoes/id/pregao/7622000000012006',
  'title': 'Pregao nº  0000012006'},
 '52131: comando da marinha': {'href': '/licitacoes/id/orgao/52131',
  'title': 'Orgão do pregão'},
 'Termos': {'href': '/pregoes/v1/termos_pregao.html?nu_pregao=12006&co_uasg=762200',
  'title': 'Termos do pregão'},
 'declaracões': {'href': '/pregoes/v1/declaracao_pregao.html?nu_pregao=12006&co_uasg=762200',
  'title': 'Declarações do pregão'},
 'itens': {'href': '/pregoes/id/pregao/7622000000012006/itens',
  'title': 'Itens do pregão'},
 'resultados': {'href': '/pregoes/v1/resultados_pregao.html?nu_pregao=12006&co_uasg=762200',
  'title': 'Resultados do pregão'},
 'uasg': {'href': '/licitacoes/id/uasg/762200',
  'title': 'UASG 762200: CENTRO DE INSTRUCAO ALMIRANTE WANDENKOLK'}}

In [4]:
df.columns

Index(['numero', 'co_portaria', 'dtPortaria', 'co_processo', 'ds_tipo_pregao',
       'ds_tipo_pregao_compra', 'tx_objeto', 'co_uasg', 'ds_situacao_pregao',
       'dtDataEdital', 'dtInicioProposta', 'dtFimProposta', '_links'],
      dtype='object')

In [5]:
df['link_pregao'] = df['_links'].apply(lambda x: url_base + x['self']['href'] + '.json')

In [6]:
df['link_itens'] = df['_links'].apply(lambda x: url_base + x['itens']['href'] + '.json') 

In [7]:
df['link_termos'] = df['_links'].apply(lambda x: url_base + x['Termos']['href'] + '.json')

In [8]:
df['link_declaracões'] = df['_links'].apply(lambda x: url_base + x['declaracões']['href'] + '.json')

In [9]:
df['link_resultados'] = df['_links'].apply(lambda x: url_base + x['resultados']['href'] + '.json')

In [10]:
for i in list(df.columns):
    if i.startswith('dt'):
        df[i] = pd.to_datetime(df[i], infer_datetime_format=True)

In [11]:
df.ds_situacao_pregao.unique()

array([None, 'revogado', 'abandonado', 'anulado', 'suspenso'],
      dtype=object)

In [12]:
df.drop(columns=['numero', '_links', 'ds_tipo_pregao'], inplace=True)
# numero: recebeu None em todas as linhas
# _links, já mapeado para as colunas link_...
# ds_tipo_pregao: todos receberam "eletrônico"


In [14]:
df.head()

,co_portaria,dtPortaria,co_processo,ds_tipo_pregao_compra,tx_objeto,co_uasg,ds_situacao_pregao,dtDataEdital,dtInicioProposta,dtFimProposta,link_pregao,link_itens,link_termos,link_declaracões,link_resultados
0,018 ...,2006-07-21,05/2006,SISPP,Objeto: Pregão Eletrônico - Aquisição de Pr ...,762200,None,2006-08-23,2006-08-23 08:00:00,2006-09-06 08:30:00,http://compras.dados.gov.br/pregoes/id/pregao/...,http://compras.dados.gov.br/pregoes/id/pregao/...,http://compras.dados.gov.br/pregoes/v1/termos_...,http://compras.dados.gov.br/pregoes/v1/declara...,http://compras.dados.gov.br/pregoes/v1/resulta...
1,018 ...,2006-07-21,762200/0001-2007,SISPP,Objeto: Pregão Eletrônico - Fornecimento de c...,762200,None,2007-02-27,2007-03-12 08:00:00,2007-03-27 09:00:00,http://compras.dados.gov.br/pregoes/id/pregao/...,http://compras.dados.gov.br/pregoes/id/pregao/...,http://compras.dados.gov.br/pregoes/v1/termos_...,http://compras.dados.gov.br/pregoes/v1/declara...,http://compras.dados.gov.br/pregoes/v1/resulta...
2,Portaria n° 42/2007 ...,2007-11-12,63093000024200771,SISRP,OBJETO : PREGAO ELETRONICO Prestaçao de serv...,762200,revogado,2008-01-18,2008-02-01 08:00:00,2008-02-15 10:00:00,http://compras.dados.gov.br/pregoes/id/pregao/...,http://compras.dados.gov.br/pregoes/id/pregao/...,http://compras.dados.gov.br/pregoes/v1/termos_...,http://compras.dados.gov.br/pregoes/v1/declara...,http://compras.dados.gov.br/pregoes/v1/resulta...
3,Portaria ...,2009-04-18,63093000001200929,SISRP,Objeto: Pregão Eletrônico - Registro de Preço...,762200,None,2009-10-22,2009-10-22 08:00:00,2009-11-09 09:30:00,http://compras.dados.gov.br/pregoes/id/pregao/...,http://compras.dados.gov.br/pregoes/id/pregao/...,http://compras.dados.gov.br/pregoes/v1/termos_...,http://compras.dados.gov.br/pregoes/v1/declara...,http://compras.dados.gov.br/pregoes/v1/resulta...
4,Portaria ...,2009-04-18,63093000116200913,SISRP,Objeto: Pregão Eletrônico - Registro de Preço...,762200,None,2010-02-01,2010-02-01 08:00:00,2010-02-11 09:30:00,http://compras.dados.gov.br/pregoes/id/pregao/...,http://compras.dados.gov.br/pregoes/id/pregao/...,http://compras.dados.gov.br/pregoes/v1/termos_...,http://compras.dados.gov.br/pregoes/v1/declara...,http://compras.dados.gov.br/pregoes/v1/resulta...


## TODO

+ Verificar o campo dtPortaria,
+ Verificar o campo ds_tipo_pregao_compra
+ Verificar o campo ds_situacao_pregao
+ Criar o banco de dados com as relações entre as informações dos links e as uasg
+ transformar numa função que receba os codigos das uasg